In [1]:
import re
from json import dump
import requests
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import urlretrieve

In [2]:
# pip install openpyxl

In [3]:
url = "https://www.australia-shoppings.com/malls-centres/victoria"
shopping_metadata = defaultdict(dict)
bs_object = BeautifulSoup(requests.get(url).text, "html.parser")

In [4]:
shopping_list = bs_object.find("ul" , {"class": "malls-list"})
# This pattern matches coordinate information
pattern = r"(-?\d+(\.\d+)?),\s*(-?\d+(\.\d+)?)"

# Loop through the list of shopping centres
for li in shopping_list.find_all("li"):
    # Extract name, address
    name = li.find("h3",{"class": "tit"}).text.strip()
    addr = li.find("p", {"class":"st"})
    # Further extract suburb and coordinate from address
    suburb = addr.text.split(',')[1]
    match = re.findall(pattern, addr.text)
    coordinate = [float(match[0][0]),float(match[0][2])]
    shopping_metadata[name] = coordinate[0], coordinate[1]

In [5]:
with open('data/shopping_centre.json', 'w') as f:
    dump(shopping_metadata, f)
shopping = pd.read_json('data/shopping_centre.json').transpose()
shopping = shopping.reset_index()
# Rename columns
shopping.columns = ['name', 'latitude', 'longitude']
shopping.to_csv('data/shopping_centre.csv', index=False)

In [6]:
output_hospital = 'data/hospitals.csv'
url = 'https://springernature.figshare.com/ndownloader/files/15591434'
urlretrieve(url, output_hospital)
hospitals = pd.read_csv("data/hospitals.csv", encoding='cp1252')
hospitals_vic = hospitals[hospitals["State"] == "Vic"]
# Extract useful features
hospitals_select = hospitals_vic[["Hospital name","Postcode","Latitude","Longitude"]].reset_index(drop=True)
hospitals_select.to_csv(output_hospital, index=False)

In [7]:
output_facility = 'data/sport_facilities.xlsx'
url = "https://discover.data.vic.gov.au/dataset/e6db797e-3801-4cfa-bf02-82350d0f722d/resource/bfff5fff-9c74-4671-8396-43f793613b70/download/srv_ifmd_all-facilities.xlsx"
urlretrieve(url, output_facility)

facility = pd.read_excel(output_facility, usecols=['Facility Name', 'Latitude', 'Longitude', 'Pcode'])
facility = facility.drop_duplicates(subset=['Facility Name']).reset_index(drop=True)
facility.to_csv('data/sport_facilities.csv')

In [8]:
output_school = 'data/schools.csv'
url = "https://www.education.vic.gov.au/Documents/about/research/datavic/dv346-schoollocations2023.csv"
urlretrieve(url, output_school)

school = pd.read_csv(output_school, usecols=['School_Name', 'Address_Postcode', 'X', 'Y'], encoding='cp1252')
school.columns = ['School_Name', 'Address_Postcode', 'Longitude', 'Latitude']
school.to_csv(output_school, index=False)

In [ ]:
output_building = 'data/buildings'